**AI & Machine Learning (KAN-CINTO4003U) - Copenhagen Business School | Spring 2025**

***

# Part I: Agents

Please see the description of the assignment in the README file (section 1) <br>
**Guide notebook**: [guides/agents_guide.ipynb](guides/agents_guide.ipynb)


***
<br>

* Remember to include some reflections on your results. Are there, for example, any hyperparameters that are particularly important?

* You should follow the steps given in the `agents_guide` notebook to create your own RAG system.

<br>

***